In [1]:
import os
import torch 
import numpy
import pandas as pd 
import cudf
import cupy
import nvtabular as nvt

### Create random input data similar to pre-processed Yoochoose dataset structure 

In [2]:
NUM_ROWS = 10000
session_length = 20
batch_size = 100
inputs = {
    'session_id': numpy.random.randint(70000, 80000, NUM_ROWS),
    'day': numpy.random.randint(1, 10, NUM_ROWS),
    'item_id': numpy.random.randint(1, 51996, NUM_ROWS),
    'category': numpy.random.randint(0, 332, NUM_ROWS),
    'timestamp/age_days': numpy.random.uniform(0, 1, NUM_ROWS),
    'timestamp/weekday/sin' : numpy.random.uniform(0, 1, NUM_ROWS),
    'purchase': numpy.random.randint(0, 2, NUM_ROWS)
    }
random_data = cudf.DataFrame(inputs)
random_data

,session_id,day,item_id,category,timestamp/age_days,timestamp/weekday/sin,purchase
0,73937,1,23547,66,0.413801,0.133115,0
1,74205,9,31135,27,0.658831,0.276254,1
2,78490,7,23840,270,0.486844,0.957908,1
3,72677,2,7843,7,0.895641,0.563399,1
4,78914,1,25748,215,0.465150,0.207356,1
...,...,...,...,...,...,...,...
9995,76882,2,4194,40,0.455592,0.415028,1
9996,78589,7,14881,268,0.517333,0.793696,0
9997,72704,5,5032,43,0.461757,0.655167,0
9998,75992,5,26427,222,0.854022,0.904570,1


### NVTabular workflow

- #TODO : Change the workflow using tagging API once it is finalized  

In [3]:
# Define Groupby Workflow
groupby_features = list(inputs.keys()) >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list"],
        "category": ["list"],     
        "day": ["first"],
        "purchase": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")
# Trim sessions to first 20 items 
groupby_features_nonlist = [x for x in groupby_features.selector if '-list' not in x]
groupby_features_nonlist
groupby_features_trim = ((groupby_features - groupby_features_nonlist)) >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')

workflow = nvt.Workflow(groupby_features + groupby_features_trim )
dataset = nvt.Dataset(random_data, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

/nvtabular/nvtabular/workflow/node.py:43: FutureWarning: The `["a", "b", "c"] >> ops.Operator` syntax for creating a `ColumnGroup` has been deprecated in NVTabular 21.09 and will be removed in a future version.
  warnings.warn(


In [4]:
sessions_gdf.head(3)

,category-list,purchase-first,timestamp/weekday/sin-list,timestamp/age_days-list,session_id,item_id-list,day-first,category-list_trim,timestamp/weekday/sin-list_trim,timestamp/age_days-list_trim,item_id-list_trim
0,"[184, 135, 224]",0,"[0.257748284548454, 0.7676927098279173, 0.7387...","[0.3356433901782223, 0.8908132518976292, 0.855...",70002,"[47039, 14928, 21948]",9,"[184, 135, 224]","[0.257748284548454, 0.7676927098279173, 0.7387...","[0.3356433901782223, 0.8908132518976292, 0.855...","[47039, 14928, 21948]"
1,[277],0,[0.2782138412720573],[0.20679906905291812],70003,[43424],2,[277],[0.2782138412720573],[0.20679906905291812],[43424]
2,[229],0,[0.2765030673053457],[0.9463081705213681],70005,[20205],1,[229],[0.2765030673053457],[0.9463081705213681],[20205]


- We can save the workflow.

In [5]:
workflow.save('workflow_inference_test')

### Export pre-processed data by day 

In [6]:
# Convert to a Dataset and write out hive-partitioned data to disk
nvt_output_path_tmp ='./output_nvt_tmp/'
PARTITION_COL = 'day-first'
nvt.Dataset(sessions_gdf).to_parquet(nvt_output_path_tmp, partition_on=[PARTITION_COL])

In [7]:
OUTPUT_FOLDER = "./preproc_sessions_by_day_ts/"
!mkdir -p $OUTPUT_FOLDER
days_folders = [f for f in sorted(os.listdir(nvt_output_path_tmp)) if f.startswith(PARTITION_COL)]
for day_folder in days_folders:
    df = cudf.read_parquet(os.path.join(nvt_output_path_tmp, day_folder))
    out_folder = os.path.join(OUTPUT_FOLDER, day_folder.replace('day-first=', ''))
    os.makedirs(out_folder, exist_ok=True)
    df.to_parquet(os.path.join(out_folder, 'train.parquet'))
    
    random_values = cupy.random.rand(len(df))
    
    #Extracts 10% for valid and test set. Those sessions are also in the train set, but as evaluation
    #happens only for the subsequent day of training, that is not an issue, and we can keep the train set larger.
    valid_set = df[random_values <= 0.10]
    valid_set.to_parquet(os.path.join(out_folder, 'valid.parquet'))
    
    test_set = df[random_values >= 0.90]
    test_set.to_parquet(os.path.join(out_folder, 'test.parquet'))

# Transformers4rec model  

In [8]:
import torch 
import transformers4rec.torch as torch4rec
from transformers4rec.torch import SequentialTabularFeatures, MLPBlock, SequentialBlock, Head, TransformerBlock

from transformers4rec.utils.schema import DatasetSchema

from transformers4rec.torch.head import NextItemPredictionTask

from transformers4rec.config import transformer
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt

- Manually set the schema 

In [9]:
# Define schema object to pass it to the SequentialTabularFeatures
schema = DatasetSchema.from_schema("schema.pb")

### Define the sequential input module

Below we define our `input` bloc using [`SequentialTabularFeatures` class](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/features/sequential.py). The `from_schema` module directly parse schema and accepts categorical and continuous sequential inputs and supports data augmentation, data aggregation, `sequential-concat` and `elementwise-sum` aggregations, the projection of the interaction embeddings and the masking tasks.

`max_sequence_length` defines the maximum sequence length of our sequential input, and if `continuous_projection` argument is set,  all numerical features are concatenated and projected by a number of MLP layers.

In [10]:
inputs = SequentialTabularFeatures.from_schema(
        schema,
        max_sequence_length=20,
        continuous_projection=64,
        d_output=100,
        masking="causal",
    )

inputs.masking.device = 'cuda'

### End-to-end session-based Transformer-based model for item prediction:

- LM task + HF Transformer architecture + Next item-prediction task. 
- We build a [T4RecConfig](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/config/transformer.py#L8) class to update the config class of the transformer architecture with the specified arguments, then load the related model. Here we use it to instantiate an XLNET model according to the  arguments (d_model, n_head, etc.), defining the model architecture.
- [TransformerBlock](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/block/transformer.py#L37) class is created to support HF Transformers for session-based and sequential-based recommendation models.
- [NextItemPredictionTask](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/head.py#L212) is the class to support next item prediction task.

In [11]:
# case-1: Define XLNetConfig class and set default parameters 

transformer_config = transformer.XLNetConfig.build(
    d_model=64, n_head=4, n_layer=2, total_seq_length=20
)

body = torch4rec.SequentialBlock(
    inputs, torch4rec.MLPBlock([64]), torch4rec.TransformerBlock(transformer=transformer_config, masking=inputs.masking)
)

head = torch4rec.Head(
    body,
    torch4rec.NextItemPredictionTask(weight_tying=True, hf_format=True, metrics=[NDCGAt(top_ks=[5,10, 1000], labels_onehot=True),
                                                                                 AvgPrecisionAt(top_ks=[5,10, 1000],labels_onehot=True)]),
    inputs=inputs,
)
model = torch4rec.Model(head)

# Train the model 

- Set the Trainer

In [12]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
#Set argumentd for training 
train_args = T4RecTrainingArguments(local_rank = -1, dataloader_drop_last = True,
                  report_to = [], debug = ["r"], gradient_accumulation_steps = 32,
                  per_device_train_batch_size = 512, per_device_eval_batch_size = 512,
                  output_dir = ".", use_legacy_prediction_loop = False)

# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    compute_metrics=True,
)

- Build dataloader 

In [13]:
import glob
from transformers4rec.torch.utils.data_utils import PyarrowDataLoaderBuilder, NVTDataLoaderBuilder
train_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/train.parquet")

# Specify sparse features
SESSION_LENGTH_MAX = 20
x_cat_names, x_cont_names = ['item_id-list_trim', 'category-list_trim'], ['timestamp/weekday/sin-list_trim','timestamp/age_days-list_trim']
sparse_features_max = {
    fname: SESSION_LENGTH_MAX
    for fname in x_cat_names + x_cont_names
}

In [14]:
pyarrow_loader = PyarrowDataLoaderBuilder(
                                    paths_or_dataset=train_data_paths, 
                                    batch_size=train_args.per_device_train_batch_size,
                                    max_sequence_length=20,
                                    dataloader_drop_last=True,
                                    shuffle=True,
                                    pin_memory=True
    
                                ).build_from_schema(schema)

nvt_loader = NVTDataLoaderBuilder(paths_or_dataset=train_data_paths, 
                                batch_size=train_args.per_device_train_batch_size,
                                max_sequence_length=20,
                                dataloader_drop_last=True,
                                shuffle=True,
                                sparse_as_dense=True,
                                sparse_names=x_cat_names+x_cont_names,
                                sparse_max=sparse_features_max
                                ).build_from_schema(schema)

- Fit the model : 

In [15]:
trainer.set_train_dataloader(nvt_loader)
trainer.reset_lr_scheduler()
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=15.08968734741211, metrics={'train_runtime': 3.0099, 'train_samples_per_second': 0.997, 'total_flos': 0.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2647527424, 'init_mem_gpu_alloc_delta': 14147584, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 28340224, 'train_mem_gpu_alloc_delta': 42939904, 'train_mem_cpu_peaked_delta': 8192, 'train_mem_gpu_peaked_delta': 440794624})

- Evaluate the model: here we evaluate the model using training set.

In [16]:
trainer.set_eval_dataloader(nvt_loader)
train_metrics = trainer.evaluate(metric_key_prefix='train')
for key in sorted(train_metrics.keys()):
    print("  %s = %s" % (key, str(train_metrics[key])))

/opt/conda/lib/python3.8/site-packages/transformers/trainer_pt_utils.py:366: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Not all data has been set. Are you sure you passed all values?
Not all data has been set. Are you sure you passed all values?


  epoch = 3.0
  eval_mem_cpu_alloc_delta = 2637824
  eval_mem_cpu_peaked_delta = 102400
  eval_mem_gpu_alloc_delta = -238592
  eval_mem_gpu_peaked_delta = 574467584
  train_avgprecisionat_@10 = 0.0
  train_avgprecisionat_@1000 = 7.8851226e-05
  train_avgprecisionat_@5 = 0.0
  train_loss = 34.1590264638265
  train_ndcgat_@10 = 0.0
  train_ndcgat_@1000 = 0.0022098827
  train_ndcgat_@5 = 0.0
  train_runtime = 24.2242
  train_samples_per_second = 422.718
